In [61]:
from __future__ import print_function

import pandas as pd
import numpy as np

from src import tracker
from genepy import terra
from genepy.utils import helper as h
from genepy.google import gcp
from genepy import sequencing as seq
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from IPython.display import Image,display
from bokeh.plotting import *

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Loading BokehJS ...

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [2]:
samplesetname = "21Q1"

workspace1="terra-broad-cancer-prod/DepMap_WGS"
workspace2="terra-broad-cancer-prod/Getz_IBM_CellLines_WGS"

refworkspace="broad-firecloud-ccle/DepMap_WGS_CN"
cgaworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline-wgs"

source1="ccle"
source2="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"

depmap_pv = "https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE"

release = samplesetname

WGSmethods = [
    "gatk/PreProcessingForVariantDiscovery_GATK4/8",
    "GP-TAG/Manta_SomaticSV/9",
    "gkugener/ArrayOfFilesToTxt/1",
    "vdauwera/BamToUnmappedRGBams/4",
    "gatk/CNV_Somatic_Pair_Workflow/9",
    "gkugener/Aggregate_CN_seg_files/2"
]

maxage='2020-09-10'

# version 102
ensemblserver = "http://nov2020.archive.ensembl.org/biomart" 

In [4]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)

refwm = dm.WorkspaceManager(refworkspace).disable_hound()
cgawm = dm.WorkspaceManager(cgaworkspace).disable_hound()

/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

In [5]:
extract_to_change = {'from_arxspan_id': 'participant'}
potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
#ccle_refsamples=pd.read_csv('temp/updated_ref_samples.csv',index_col=0)

depmap_pv = sheets.get(depmap_pv).sheets[0].to_frame(header=2)
depmap_pv = depmap_pv.drop(depmap_pv.iloc[:1].index).set_index('DepMap_ID', drop=True)[['Culture Type', 'Culture Medium']]
depmap_master = tc.get(name='arxspan-cell-line-export-f808', file='ACH').set_index('arxspan_id',drop=True)
depmap_master = depmap_master.join(depmap_pv)

[##################]100% |  18.2 MiB/s |   1.0 MiB /   1.0 MiB | Time:  0:00:00


In [133]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)


In [9]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")
a=ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ["stripped_cell_name", "participant_id", "cellosaurus_id", "internal_bam_filepath", "internal_bai_filepath", "legacy_bam_filepath", "legacy_bai_filepath", "parent_cell_line", "sex", "matched_normal", "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "mediatype", "condition", "baits", "source", "sequencing_date", 'size', 'crc32c_hash', 'md5_hash', 'legacy_size', "legacy_crc32c_hash"]]
a.index.name="sample_id"
a = a.rename(columns={'patient_id': 'participant_id'})
wm.upload_samples(a)

Successfully imported 3 participants.
Successfully imported 3 samples.


In [10]:
wm.update_sample_set('toremap', a.index.tolist())

Successfully imported 1 sample sets:
  * toremap (3 samples)


In [14]:
wm.create_submission('star_v1-0_BETA_cfg', "toremap", "sample_set", "this.samples")

Successfully created submission 110b9f37-9430-40ef-ac0a-9c7daf60b898.


'110b9f37-9430-40ef-ac0a-9c7daf60b898'

In [15]:
sam= wm.get_samples()

In [17]:
sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']]

,star_bam_file,star_bam_index
sample_id,,
CDS-G0F5f5,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...
CDS-kU30H5,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...
CDS-fk564T,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...


In [18]:
a=terra.changeToBucket(sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']],'gs://cclebams/rnasq_hg38/', name_col= "index" , values=['star_bam_file','star_bam_index'], filetypes=['bam', 'bai'], catchdup=True, test=False)

In [26]:
ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['internal_bam_filepath', 'internal_bai_filepath']]

,internal_bam_filepath,internal_bai_filepath
CDS-G0F5f5,gs://cclebams/rnasq_hg38/CDS-G0F5f5.bam,gs://cclebams/rnasq_hg38/CDS-G0F5f5.bai
CDS-kU30H5,gs://cclebams/rnasq_hg38/CDS-kU30H5.bam,gs://cclebams/rnasq_hg38/CDS-kU30H5.bai
CDS-fk564T,gs://cclebams/rnasq_hg38/CDS-fk564T.bam,gs://cclebams/rnasq_hg38/CDS-fk564T.bai


In [108]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

## PART 2

In [193]:
a = ! gsutil ls gs://cclebams/rnasq_hg38/

In [201]:
set(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index) - set(["CDS"+i.split('/CDS')[-1].split('.Ali')[0] for i in a])

{'CDS-ABH0uZ', 'CDS-G0F5f5', 'CDS-fk564T', 'CDS-kU30H5'}

In [183]:
l

['gs://cclebams/rna/921.bai',
 'gs://cclebams/rna/921.bam',
 'gs://cclebams/rna/94T778.bai',
 'gs://cclebams/rna/95T1000.bai',
 'gs://cclebams/rna/95T1000.bam',
 'gs://cclebams/rna/95T1000.bam.md5',
 'gs://cclebams/rna/A375SKINCJ1.bai',
 'gs://cclebams/rna/A375SKINCJ1.bam.md5',
 'gs://cclebams/rna/A375SKINCJ2.bai',
 'gs://cclebams/rna/A375SKINCJ2.bam',
 'gs://cclebams/rna/A375SKINCJ2.bam.md5',
 'gs://cclebams/rna/A375SKINCJ3.bai',
 'gs://cclebams/rna/A375SKINCJ3.bam.md5',
 'gs://cclebams/rna/A388.bai',
 'gs://cclebams/rna/ACxtgE_M040416.bai',
 'gs://cclebams/rna/ACxtgE_M040416.bam',
 'gs://cclebams/rna/AfBxpu_RMSYM.bai',
 'gs://cclebams/rna/AfBxpu_RMSYM.bam',
 'gs://cclebams/rna/BPH1.bai',
 'gs://cclebams/rna/Becker.bai',
 'gs://cclebams/rna/C396.bai',
 'gs://cclebams/rna/C396.bam.md5',
 'gs://cclebams/rna/CASKI.bai',
 'gs://cclebams/rna/CCLFUPGI0005T.bai',
 'gs://cclebams/rna/CCLP1.bai',
 'gs://cclebams/rna/CCLP1.bam.md5',
 'gs://cclebams/rna/CCSW1.bai',
 'gs://cclebams/rna/CCSW1.bam'

In [504]:
i  = ! gsutil ls gs://cclebams/**

In [449]:
b = [a for a in i if a.endswith('.bam')]

In [469]:
len(b)

2122

In [494]:
bi = [a for a in i if a.endswith('.bam') and a[:-4]+".bai" not in i and a+".bai" not in i]

In [497]:
len(bi)

41

In [511]:
! gsutil ls gs://cclebams/wes/Getz_IBM_DMX_CellLines_12samples_deepcov_01302018/RP-1561/Exome/RVH421SKINFV1/v1/RVH421SKINFV1.bam

CommandException: One or more URLs matched no objects.


In [513]:
u = set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath)- set([a for a in i if a.endswith('.bam')])

In [526]:
sizes = [gcp.extractSize(a) for a in gcp.lsFiles(set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath), "-l")]

listing files in gs


In [527]:
sizes

[('gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam', 26288709027),
 ('gs://cclebams/wes/CDS-2JxT1P.bam', 17135716554),
 ('gs://cclebams/wes/CDS-OLgoE4.bam', 31217641284),
 ('gs://cclebams/wes/CDS-Ql8GJZ.bam', 27878740151),
 ('gs://cclebams/wes/CDS-wlTAAF.bam', 25252308262),
 ('gs://cclebams/wes/CDS-lTogDX.bam', 26167931603),
 ('gs://cclebams/wes/CDS-9zidMf.bam', 10570660641),
 ('gs://cclebams/wes/CDS-NPG23x.bam', 21689262653),
 ('gs://cclebams/wes/CDS-6EyvRQ.bam', 27725567504),
 ('gs://cclebams/wes/RP-1561/Exome/SK-MEL-2/current/SK-MEL-2.bam',
  28757788344),
 ('gs://cclebams/wes/CDS-PHI8VT.bam', 22496779642),
 ('gs://cclebams/wes/CDS-Y27yfi.bam', 26154087701),
 ('gs://cclebams/wes/CDS-Pkk9e2.bam', 23213429069),
 ('gs://cclebams/wes/CDS-IzeN7a.bam', 22051020828),
 ('gs://cclebams/wes/CDS-Md89va.bam', 21788262259),
 ('gs://cclebams/wes/CDS-ohjYlg.bam', 23932006016),
 ('gs://cclebams/wes/CDS-G1sVsw.bam', 17999530720),
 ('gs://cclebams/wes/CDS-wbPtTZ.bam', 26597306271),
 ('gs:

In [532]:
torm = []
tofind = []
for val in sizes:
    a = ccle_refsamples[ccle_refsamples.legacy_size==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    a = ccle_refsamples[ccle_refsamples['size']==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    tofind.append(val[0])

In [535]:
! gsutil ls -l gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam

26288709027  2020-03-09T15:43:34Z  gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam
TOTAL: 1 objects, 26288709027 bytes (24.48 GiB)


In [537]:
! gsutil ls -l gs://cclebams/wes/CDS-CmkOx5.bam

24761705198  2020-09-24T14:22:02Z  gs://cclebams/wes/CDS-CmkOx5.bam
TOTAL: 1 objects, 24761705198 bytes (23.06 GiB)


In [539]:
! gsutil ls -l gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam

24761705198  2020-03-09T15:42:51Z  gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam
TOTAL: 1 objects, 24761705198 bytes (23.06 GiB)


## PART 3

In [5]:
## missing legacy for rna/wes
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bam_filepath.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [6]:
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bai_filepath.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [6]:
## wrong filepath legacy for rna/wes
miss_rwbi = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bai_filepath.tolist())

listing files in gs


In [7]:
miss_rw = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bam_filepath.tolist())

listing files in gs


In [8]:
## which ones can we get from remapping
remapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ~ccle_refsamples.internal_bam_filepath.isna()]
len(remapable)

156

In [12]:
remapable[remapable.datatype=='wes']

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-fjMmZs,ACH-001529,2,NaN,NaN,wes,1.925572e+10,NaN,JAR,PT-4VYF6QwQ,CVCL_0360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-i7LdYv,ACH-000051,1,NaN,NaN,wes,1.061912e+10,NaN,TE617T,PT-2idRHKaT,CVCL_1755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-M33guL,ACH-000032,1,NaN,NaN,wes,4.340937e+09,NaN,MHHCALL3,PT-jgh3Ohro,CVCL_0089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-tfg4Dr,ACH-001282,1,NaN,NaN,wes,7.107219e+09,NaN,CHLA258,PT-PiD3Gm5d,CVCL_A058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-CAB69b,ACH-000728,1,NaN,NaN,wes,4.291779e+09,NaN,KASUMI2,PT-MS6R4JQq,CVCL_0590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-clTgER,ACH-000577,2,NaN,NaN,wes,2.511173e+10,NaN,JHH2,PT-R2lQDrhy,CVCL_2786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-wXWgLf,ACH-000596,2,NaN,NaN,wes,1.983989e+10,NaN,LCLC97TM1,PT-MgrYS90q,CVCL_1376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-GTkHGf,ACH-001075,2,NaN,NaN,wes,4.387466e+10,NaN,NCIH292,PT-0IujQn8b,CVCL_0455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-COhPnV,ACH-000800,2,NaN,NaN,wes,6.522894e+10,NaN,NCIH446,PT-bRUartb7,CVCL_1562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [14]:
## remapping
weswm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")

#weswm.update_sample_set("toremap", remapable.index.tolist())

In [19]:
a = weswm.create_submission('BamToUnmappedRGBams_MC', "toremap", "sample_set", "this.samples")

Successfully created submission cdd5a065-d28d-4750-9417-efae23e5e893.


TypeError: waitForSubmission() missing 1 required positional argument: 'submissions'

In [32]:
a = weswm.create_submission('Generate_uBAM_File_List', "toremap", "sample_set", "this.samples")

/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Successfully created submission 34b65cef-361e-420f-b323-94cc52ae0170.


/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

In [73]:
a = weswm.create_submission('Realign_WES_GATK4_hg19', "toremap", "sample_set", "this.samples")

Successfully created submission 61d87541-d51b-4de0-b9ac-847ce2106d08.


In [557]:
## the ones we can't
nonremapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ccle_refsamples.internal_bam_filepath.isna()]
nonremapable

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [59]:
## indexing
a, missrwbi = gcp.exists(ccle_refsamples[~ccle_refsamples.legacy_bai_filepath.isna()].legacy_bai_filepath.tolist())

listing files in gs


In [114]:
a, missrw = gcp.exists(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna()].legacy_bam_filepath.tolist())

listing files in gs


In [62]:
missrwbirwbi

{'gs://cclebams/hg38_wgs/CDS-8Ut3sT.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-Ip02tY.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-Rd4nMx.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-TGTiB8.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-VnMBYD.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-YSRYLi.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-ZJh6UN.hg38.bai',
 'gs://cclebams/hg38_wgs/CDS-dgxjAa.hg38.bai',
 'gs://cclebams/wes/143B.bai',
 'gs://cclebams/wes/C817/EWS-502/current/EWS-502.bai',
 'gs://cclebams/wes/C836.BICR_18.2.bai',
 'gs://cclebams/wes/C836.CL-14.2.bai',
 'gs://cclebams/wes/C836.HCC-366.2.bai',
 'gs://cclebams/wes/C836.HEC-50B.1.bai',
 'gs://cclebams/wes/C836.HEC-59.2.bai',
 'gs://cclebams/wes/C836.Hs_255.T.2.bai',
 'gs://cclebams/wes/C836.Hs_611.T.2.bai',
 'gs://cclebams/wes/C836.Hs_895.T.1.bai',
 'gs://cclebams/wes/C836.KG-1-C.1.bai',
 'gs://cclebams/wes/C836.KMRC-3.2.bai',
 'gs://cclebams/wes/C836.KMS-11.2.bai',
 'gs://cclebams/wes/C836.NCI-H1339.2.bai',
 'gs://cclebams/wes/C836.NCI-H1436.3.bai',
 'gs

In [ ]:
bais = await seq.indexBams(bams=ccle_refsamples[(~ccle_refsamples.legacy_bai_filepath.isna()) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath, cores=12)

In [69]:
bais = ! gsutil ls gs://cclebams/**.bai

In [75]:
ccle_refsamples.loc[ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].index, 'legacy_bai_filepath'] = [i[:-4]+".bai" if i[:-4]+".bai" in bais else None for i in ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath]

In [89]:
gcp.rmFiles(set(bais) - (set(ccle_refsamples.internal_bai_filepath) | set(ccle_refsamples.legacy_bai_filepath)), dryrun=False)

In [86]:
len(set(ccle_refsamples.internal_bai_filepath) | set(ccle_refsamples.legacy_bai_filepath))

10711

In [83]:
ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath

CDS-d2xe4x                gs://cclebams/wes/SANGER_C32_SKIN.bam
CDS-ezLdbO    gs://cclebams/wes/SANGER_HN_UPPER_AERODIGESTIV...
CDS-NE61ys           gs://cclebams/wes/SANGER_NCIH1755_LUNG.bam
CDS-TGTiB8           gs://cclebams/hg38_wgs/CDS-TGTiB8.hg38.bam
CDS-dgxjAa           gs://cclebams/hg38_wgs/CDS-dgxjAa.hg38.bam
CDS-8Ut3sT           gs://cclebams/hg38_wgs/CDS-8Ut3sT.hg38.bam
CDS-Ip02tY           gs://cclebams/hg38_wgs/CDS-Ip02tY.hg38.bam
CDS-YSRYLi           gs://cclebams/hg38_wgs/CDS-YSRYLi.hg38.bam
CDS-ZJh6UN           gs://cclebams/hg38_wgs/CDS-ZJh6UN.hg38.bam
CDS-Rd4nMx           gs://cclebams/hg38_wgs/CDS-Rd4nMx.hg38.bam
CDS-VnMBYD           gs://cclebams/hg38_wgs/CDS-VnMBYD.hg38.bam
Name: legacy_bam_filepath, dtype: object

In [82]:
a = await seq.indexBams(ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath, cores=8)

ValueError: issue with the command: b'[E::hts_open_format] Failed to open file gs://cclebams/hg38_wgs/CDS-TGTiB8.hg38.bam\nsamtools index: failed to open "gs://cclebams/hg38_wgs/CDS-TGTiB8.hg38.bam"[E::hts_open_format] : No such file or directory\nFailed to open file gs://cclebams/wes/SANGER_HN_UPPER_AERODIGESTIVE_TRACT.bam\nsamtools index: failed to open "gs://cclebams/wes/SANGER_HN_UPPER_AERODIGESTIVE_TRACT.bam": No such file or directory\n[E::hts_open_format] Failed to open file gs://cclebams/hg38_wgs/CDS-Ip02tY.hg38.bam\nsamtools index: failed to open "gs://cclebams/hg38_wgs/CDS-Ip02tY.hg38.bam": No such file or directory\n[E::hts_open_format] Failed to open file gs://cclebams/hg38_wgs/CDS-8Ut3sT.hg38.bam\nsamtools index: failed to open "gs://cclebams/hg38_wgs/CDS-8Ut3sT.hg38.bam": No such file or directory\n[E::hts_open_format] Failed to open file gs://cclebams/hg38_wgs/CDS-dgxjAa.hg38.bam\nsamtools index: failed to open "gs://cclebams/hg38_wgs/CDS-dgxjAa.hg38.bam": No such file o

In [73]:
ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath

CDS-d2xe4x                gs://cclebams/wes/SANGER_C32_SKIN.bam
CDS-ezLdbO    gs://cclebams/wes/SANGER_HN_UPPER_AERODIGESTIV...
CDS-NE61ys           gs://cclebams/wes/SANGER_NCIH1755_LUNG.bam
CDS-C95iHs                     gs://cclebams/wes/CDS-M8aV3P.bam
CDS-qrjNBy                     gs://cclebams/wes/CDS-WHZolj.bam
CDS-q0PJc9                     gs://cclebams/wes/CDS-6mq2Or.bam
CDS-ixXfW6                     gs://cclebams/wes/CDS-UxKEaK.bam
CDS-XL3EP1                     gs://cclebams/wes/CDS-UfC2Dz.bam
CDS-MQDYm1                     gs://cclebams/wes/CDS-H1oKTL.bam
CDS-7oAFaO                     gs://cclebams/wes/CDS-LCfY0q.bam
CDS-TGTiB8           gs://cclebams/hg38_wgs/CDS-TGTiB8.hg38.bam
CDS-dgxjAa           gs://cclebams/hg38_wgs/CDS-dgxjAa.hg38.bam
CDS-o6r3vt      gs://cclebams/wgs/G16350/DMS_273/v4/DMS_273.bam
CDS-8Ut3sT           gs://cclebams/hg38_wgs/CDS-8Ut3sT.hg38.bam
CDS-Ip02tY           gs://cclebams/hg38_wgs/CDS-Ip02tY.hg38.bam
CDS-YSRYLi           gs://cclebams/hg38_

In [107]:
## recompute size/ hashes for rna/wes legacy
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].index, "legacy_crc32c_hash"] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].legacy_bam_filepath, "-L")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].index, "legacy_size"] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].legacy_bam_filepath, "-l")]

listing files in gs
listing files in gs


In [11]:
## missing bam filepath 
missing = ccle_refsamples[ccle_refsamples.internal_bam_filepath.isna()]
missing

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [60]:
## missing bai filepath
missing = ccle_refsamples[ccle_refsamples.internal_bai_filepath.isna()]
missing

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [562]:
## wrong bam filepath
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath.tolist())

listing files in gs


In [565]:
wrongfilepath

(True, set())

In [13]:
## recompute size/hashes for bam filepath
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "crc32c_hash"] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "size"] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-l")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "md5_hash"] = [gcp.extractHash(i, "md5") for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]

listing files in gs
listing files in gs
listing files in gs


In [120]:
## duplicate size accross legacy/internal 
h.dups(ccle_refsamples['legacy_size'].tolist())

[21653402880.0,
 8323552897.0,
 77095901954.0,
 10219429250.0,
 15536656644.0,
 21375250186.0,
 26384397071.0,
 8064961937.0,
 20050486293.0,
 39118676511.0,
 20926783521.0,
 14152939691.0,
 11540832045.0,
 10973551792.0,
 152230340144.0,
 23522929849.0,
 19734437052.0,
 12697329597.0,
 17504102082.0,
 5187776580.0,
 16936446279.0,
 16782522569.0,
 16623746126.0,
 14589983567.0,
 10631884636.0,
 9033346659.0,
 11199852012.0,
 33749561325.0,
 17101476207.0,
 11582467695.0,
 55034600435.0,
 62281306490.0,
 15901516795.0]

In [117]:
ccle_refsamples[ccle_refsamples['size'].isin([10555223974.0, 5176380746.0, 6707901359.0, 4634241136.0, 8984432599.0])].internal_bam_filepath

CDS-aZKAeT    gs://cclebams/hg38_wes/CDS-aZKAeT.hg38.bam
CDS-HbNAQK    gs://cclebams/hg38_wes/CDS-HbNAQK.hg38.bam
CDS-z1NQiF    gs://cclebams/hg38_wes/CDS-z1NQiF.hg38.bam
CDS-UQhyV8    gs://cclebams/hg38_wes/CDS-UQhyV8.hg38.bam
CDS-0tKUL7    gs://cclebams/hg38_wes/CDS-0tKUL7.hg38.bam
CDS-GuKG2u    gs://cclebams/hg38_wes/CDS-GuKG2u.hg38.bam
CDS-gLz8Kz    gs://cclebams/hg38_wes/CDS-gLz8Kz.hg38.bam
CDS-T5BcdG    gs://cclebams/hg38_wes/CDS-T5BcdG.hg38.bam
CDS-phR2eo    gs://cclebams/hg38_wes/CDS-phR2eo.hg38.bam
CDS-gJupgp    gs://cclebams/hg38_wes/CDS-gJupgp.hg38.bam
Name: internal_bam_filepath, dtype: object

In [143]:
remove = ['CDS-T5BcdG','CDS-gJupgp',"CDS-phR2eo","CDS-GuKG2u", "CDS-gLz8Kz",        "CDS-NUlX3d","CDS-b5ElTm","CDS-ljFuDX", "CDS-VS9XDY", "CDS-bb2V33", "CDS-lIXOWR","CDS-k9Qfva","CDS-9TDRpv","CDS-pBashm","CDS-6i6dRP","CDS-yu1s5X","CDS-g2J7MD","CDS-oRM8DN","CDS-2jBQ8n","CDS-c51IFr","CDS-59uKc2","CDS-363TYH","CDS-IQuj9W","CDS-R22IHj","CDS-up4Vo5","CDS-jSUD4f","CDS-RLVrVE","CDS-5x4qLj","CDS-6YFjST","CDS-2HO10g","CDS-vrqu12","CDS-pIy1EQ"]

In [151]:
ccle_refsamples = ccle_refsamples[~ccle_refsamples.index.isin(['CDS-BNyMCM'])]

In [142]:
,

('CDS-6Je1tY',)

In [154]:
dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38").delete_sample(set(["CDS-BNyMCM"]+remove) & set(dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38").get_samples()))

Sample(s) set() successfully deleted.


In [153]:
terra.removeSamples("broad-firecloud-ccle/DepMap_WES_CN_hg38",["CDS-BNyMCM"]+remove)

we had pairs.
Pair set "20Q2" (8 pairs) successfully updated.
Pair set "21Q1" (1 pairs) successfully updated.
Pair set "all" (1937 pairs) successfully updated.
Pair set "all_agilent" (1392 pairs) successfully updated.
Pair set "all_ice" (544 pairs) successfully updated.
Pair set "withNormal" (33 pairs) successfully updated.
Pair(s) ['CDS-2HO10g_nan', 'CDS-2jBQ8n_nan', 'CDS-363TYH_nan', 'CDS-59uKc2_nan', 'CDS-5x4qLj_nan', 'CDS-6i6dRP_nan', 'CDS-6YFjST_nan', 'CDS-9TDRpv_nan', 'CDS-b5ElTm_nan', 'CDS-bb2V33_nan', 'CDS-BNyMCM_nan', 'CDS-c51IFr_nan', 'CDS-g2J7MD_nan', 'CDS-gJupgp_nan', 'CDS-gLz8Kz_nan', 'CDS-GuKG2u_nan', 'CDS-IQuj9W_nan', 'CDS-jSUD4f_nan', 'CDS-k9Qfva_nan', 'CDS-lIXOWR_nan', 'CDS-ljFuDX_nan', 'CDS-NUlX3d_nan', 'CDS-oRM8DN_nan', 'CDS-pBashm_nan', 'CDS-phR2eo_nan', 'CDS-pIy1EQ_nan', 'CDS-RLVrVE_nan', 'CDS-T5BcdG_nan', 'CDS-up4Vo5_nan', 'CDS-vrqu12_nan', 'CDS-VS9XDY_nan', 'CDS-yu1s5X_nan'] successfully deleted.


APIException: Unable to delete samples: (400) : {
  "causes": [],
  "message": "Entities were not found:\nsample CDS-R22IHj does not exist in broad-firecloud-ccle/DepMap_WES_CN_hg38",
  "source": "rawls",
  "stackTrace": [],
  "statusCode": 400,
  "timestamp": 1616008930114
}

In [150]:
ccle_refsamples[ccle_refsamples['legacy_size'].isin([19734437052.0])][['arxspan_id','version',"datatype","size","legacy_size"]].sort_values(by="legacy_size").iloc[:40]

,arxspan_id,version,datatype,size,legacy_size
CDS-w6Ltgo,ACH-003000,1,wes,1.384567e+10,1.973444e+10
CDS-BNyMCM,ACH-002446,2,wes,1.384567e+10,1.973444e+10


In [158]:
for val in h.dups(list(set(ccle_refsamples['size']))+list(set(ccle_refsamples['legacy_size']))):
    print(ccle_refsamples[(ccle_refsamples['size']==val)|(ccle_refsamples['legacy_size']==val)][['arxspan_id','version',"datatype","size","legacy_size"]])

            arxspan_id  version datatype          size   legacy_size
CDS-hPRaPF  ACH-001694        1      wgs  1.083189e+11  1.083189e+11
            arxspan_id  version datatype          size   legacy_size
CDS-uGV4uK  ACH-000612        1      wgs  1.395618e+11  1.395618e+11
            arxspan_id  version datatype          size   legacy_size
CDS-OxwNRn  ACH-002516        1      wgs  1.187799e+11  1.187799e+11
            arxspan_id  version datatype          size   legacy_size
CDS-nru2hL  ACH-000509        1      wgs  9.799055e+10  9.799055e+10
            arxspan_id  version datatype          size   legacy_size
CDS-I5voud  ACH-002465        1      wgs  1.037878e+11  1.037878e+11
            arxspan_id  version datatype          size   legacy_size
CDS-uENFGc  ACH-002544        1      wgs  1.196478e+11  1.196478e+11
            arxspan_id  version datatype          size   legacy_size
CDS-YCehGZ  ACH-001097        1      wgs  9.978713e+10  9.978713e+10
            arxspan_id  version da

In [14]:
## missingbam index
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bai_filepath.isna()].internal_bai_filepath.tolist())

listing files in gs


In [24]:
## bam index non matching anything
bais= await seq.indexBams(bams= ccle_refsamples[ccle_refsamples.internal_bai_filepath.isin(wrongfilepath[1])].internal_bam_filepath, bucketpath=None, cores=10)

In [27]:
for k,val in bais.items():
    ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.internal_bam_filepath == k].index, 'internal_bai_filepath'] = val

In [159]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [109]:
i  = ! gsutil ls gs://cclebams/**

In [110]:
## bam non matching anything in the sample tracker
nonmatch = set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples.internal_bam_filepath.tolist()+ ccle_refsamples.legacy_bam_filepath.tolist())
nonmatch

{'gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001533_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001574_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001678_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001711_3.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001847_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002021_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002048_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002065_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-003000_1.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0aJ4Yh.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0pZb0j.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0qPmaJ.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1PXzlf.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1b1Hxk.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1djAlo.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1p2nnc.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1uWUTi.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-294bk6.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-2JxT1P.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-2Q2Kia.hg38

In [111]:
## bam that match something in size and can be deleted or can replace the other one
sizze=[gcp.extractSize(i) for i in gcp.lsFiles(nonmatch, '-l')]

listing files in gs


In [46]:
sizze

[('gs://cclebams/hg38_wes/CDS-ydPJEM.hg38.bam', 33590349704),
 ('gs://cclebams/hg38_wes/CDS-s7pOQR.hg38.bam', 52259732517),
 ('gs://cclebams/wes/CDS-eGQYXr.bam', 24098037187),
 ('gs://cclebams/hg38_wes/CDS-Pkk9e2.hg38.bam', 16985791742),
 ('gs://cclebams/wes/CDS-Fz0HXE.bam', 25843558785),
 ('gs://cclebams/hg38_wes/CDS-soTPPi.hg38.bam', 15724674947),
 ('gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam', 18181200052),
 ('gs://cclebams/wes/CDS-4S6juQ.bam', 29576566390),
 ('gs://cclebams/wes/CDS-UL1jLm.bam', 25516864764),
 ('gs://cclebams/rna/UPCISCC111.bam', 3821164738),
 ('gs://cclebams/hg38_wes/CDS-KgRznV.hg38.bam', 16089050631),
 ('gs://cclebams/hg38_wes/CDS-txTRwz.hg38.bam', 13238732111),
 ('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217),
 ('gs://cclebams/hg38_wes/CDS-zwAn7G.hg38.bam', 20340862682),
 ('gs://cclebams/hg38_wes/CDS-fs8moU.hg38.bam', 21098867720),
 ('gs://cclebams/wes/CDS-gKIdjs.bam', 16134677521),
 ('gs://cclebams/hg38_wes/CDS-kAARUi.hg38.bam', 1877

In [52]:
miss_rw

(False,
 {'gs://cclebams/wes/C817/CHLA-258/current/CHLA-258.bam',
  'gs://cclebams/wes/C836.253J.1.bam',
  'gs://cclebams/wes/C836.GDM-1.2.bam',
  'gs://cclebams/wes/C836.GRANTA-519.2.bam',
  'gs://cclebams/wes/C836.HMC-1-8.2.bam',
  'gs://cclebams/wes/C836.JJN-3.1.bam',
  'gs://cclebams/wes/C836.JMSU-1.1.bam',
  'gs://cclebams/wes/C836.KASUMI-2.1.bam',
  'gs://cclebams/wes/C836.KMM-1.1.bam',
  'gs://cclebams/wes/C836.KMS-12-BM.1.bam',
  'gs://cclebams/wes/C836.KMS-20.1.bam',
  'gs://cclebams/wes/C836.L3.3.1.bam',
  'gs://cclebams/wes/C836.LMSU.2.bam',
  'gs://cclebams/wes/C836.MFE-296.2.bam',
  'gs://cclebams/wes/C836.MHH-CALL-2.1.bam',
  'gs://cclebams/wes/C836.MHH-CALL-3.1.bam',
  'gs://cclebams/wes/C836.NCI-H1341.2.bam',
  'gs://cclebams/wes/C836.NCI-H1373.1.bam',
  'gs://cclebams/wes/C836.OVTOKO.2.bam',
  'gs://cclebams/wes/C836.PLC_PRF_5.2.bam',
  'gs://cclebams/wes/C836.RD-ES.2.bam',
  'gs://cclebams/wes/C836.SNU-175.2.bam',
  'gs://cclebams/wes/C836.T3M-4.2.bam',
  'gs://ccleba

In [112]:
found=[]
torm=[]
remaining = set()
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['legacy_size']==val[1])]
    if len(res) >0:
        if res['legacy_bam_filepath'].values[0] in miss_rw[1]:
            print("found!-----------")
            ccle_refsamples.loc[res.index, 'legacy_bam_filepath'] = val[0]
            found.append(res.index[0])
        else:
            torm.append(val[0])
    else:
        remaining.add(val[0])

NameError: name 'miss_rw' is not defined

In [69]:
found

['CDS-C95iHs',
 'CDS-qrjNBy',
 'CDS-ixXfW6',
 'CDS-TOM1NB',
 'CDS-pQE1r4',
 'CDS-7oAFaO',
 'CDS-MQDYm1',
 'CDS-FNFqnr',
 'CDS-elPhWo',
 'CDS-rG42Rn',
 'CDS-C9v2MI',
 'CDS-Oidz6E',
 'CDS-1xwlXO',
 'CDS-uMD8jg',
 'CDS-XL3EP1',
 'CDS-oLN6S3',
 'CDS-fxAhOu',
 'CDS-q0PJc9',
 'CDS-kpEDhO',
 'CDS-TsX9vB']

In [9]:
torm = ['gs://cclebams/wes/CDS-eGQYXr.bam',
 'gs://cclebams/wes/CDS-Fz0HXE.bam',
 'gs://cclebams/wes/CDS-4S6juQ.bam',
 'gs://cclebams/wes/CDS-UL1jLm.bam',
 'gs://cclebams/rna/UPCISCC111.bam',
 'gs://cclebams/wes/CDS-gKIdjs.bam',
 'gs://cclebams/rna/TERA2.bam',
 'gs://cclebams/wes/CDS-Hx6zuD.bam',
 'gs://cclebams/wes/CDS-Xgu4mi.bam',
 'gs://cclebams/wes/CDS-kt2Gne.bam',
 'gs://cclebams/wes/CDS-SvzhGj.bam',
 'gs://cclebams/wes/CDS-Pku96X.bam',
 'gs://cclebams/wes/CDS-leyYAD.bam',
 'gs://cclebams/wes/CDS-R6ehaT.bam',
 'gs://cclebams/wes/CDS-cKMeDY.bam',
 'gs://cclebams/wes/CDS-I7bMcd.bam',
 'gs://cclebams/wes/CDS-GnBdHN.bam',
 'gs://cclebams/wes/CDS-PHI8VT.bam',
 'gs://cclebams/wes/CDS-Y27yfi.bam',
 'gs://cclebams/wes/CDS-sieIuO.bam',
 'gs://cclebams/rna/MM253.bam',
 'gs://cclebams/wes/CDS-3mvYnW.bam',
 'gs://cclebams/rna/PEDS117.bam',
 'gs://cclebams/wes/CDS-AOWMF3.bam',
 'gs://cclebams/wes/CDS-8z476r.bam',
 'gs://cclebams/wes/CDS-qaOoHQ.bam',
 'gs://cclebams/wes/CDS-xCyamv.bam',
 'gs://cclebams/wes/CDS-6PZKz8.bam',
 'gs://cclebams/wes/CDS-N83rwD.bam',
 'gs://cclebams/wes/CDS-5bQzF2.bam',
 'gs://cclebams/wes/CDS-3pZIvU.bam',
 'gs://cclebams/wes/CDS-R3txwY.bam',
 'gs://cclebams/wes/CDS-O8dfj7.bam',
 'gs://cclebams/wes/CDS-D6mIfI.bam',
 'gs://cclebams/wes/CDS-9sg0Pm.bam',
 'gs://cclebams/wes/CDS-C7o0op.bam',
 'gs://cclebams/wes/CDS-aDUHcI.bam',
 'gs://cclebams/wes/CDS-YnodyM.bam',
 'gs://cclebams/wes/CDS-s7pOQR.bam',
 'gs://cclebams/wes/CDS-V6Kk5q.bam',
 'gs://cclebams/wes/CDS-WP95Oi.bam',
 'gs://cclebams/wes/CDS-HkZUmY.bam',
 'gs://cclebams/wes/CDS-rVAuin.bam',
 'gs://cclebams/wes/CDS-LifesX.bam',
 'gs://cclebams/wes/CDS-NBnCDl.bam',
 'gs://cclebams/wes/CDS-J6kDsZ.bam',
 'gs://cclebams/wes/CDS-LOW19e.bam',
 'gs://cclebams/wes/CDS-h4mOdz.bam',
 'gs://cclebams/wes/CDS-9u5DMn.bam',
 'gs://cclebams/wes/CDS-rQMY3G.bam',
 'gs://cclebams/wes/CDS-9M8GNS.bam',
 'gs://cclebams/wes/CDS-wSV3OM.bam',
 'gs://cclebams/wes/CDS-4BrJr7.bam',
 'gs://cclebams/wes/CDS-sbwn0P.bam',
 'gs://cclebams/wes/CDS-WAPQGk.bam',
 'gs://cclebams/wes/CDS-tORJC8.bam',
 'gs://cclebams/rna/SNU2372.bam',
 'gs://cclebams/wes/CDS-CZstO2.bam',
 'gs://cclebams/wes/CDS-bdb5iE.bam',
 'gs://cclebams/rna/SUM190PT.bam',
 'gs://cclebams/wes/CDS-opnGD7.bam',
 'gs://cclebams/wes/CDS-c2Sowd.bam',
 'gs://cclebams/wes/CDS-NPG23x.bam',
 'gs://cclebams/wes/CDS-w7i5l7.bam',
 'gs://cclebams/wes/CDS-iRstNJ.bam',
 'gs://cclebams/rna/HEMCSS.bam',
 'gs://cclebams/wes/CDS-ZGlgTf.bam',
 'gs://cclebams/wes/CDS-ohjYlg.bam',
 'gs://cclebams/wes/CDS-RWYJ02.bam',
 'gs://cclebams/wes/CDS-ogUnWk.bam',
 'gs://cclebams/wes/CDS-ktRRkc.bam',
 'gs://cclebams/wes/CDS-Qbfoau.bam',
 'gs://cclebams/wes/CDS-m49nRz.bam',
 'gs://cclebams/wes/CDS-P79y6z.bam',
 'gs://cclebams/wes/CDS-lTogDX.bam',
 'gs://cclebams/wes/CDS-iX8vqU.bam',
 'gs://cclebams/wes/CDS-5hbofu.bam',
 'gs://cclebams/wes/CDS-nTW67d.bam',
 'gs://cclebams/wes/CDS-ldrQm3.bam',
 'gs://cclebams/wes/CDS-xI8ZAZ.bam',
 'gs://cclebams/rna/VAL.bam',
 'gs://cclebams/wes/CDS-UvBswk.bam',
 'gs://cclebams/wes/CDS-HjGCvC.bam',
 'gs://cclebams/rna/UPCISCC072.bam',
 'gs://cclebams/wes/CDS-YYd4ww.bam',
 'gs://cclebams/wes/CDS-IzeN7a.bam',
 'gs://cclebams/wes/CDS-2JxT1P.bam',
 'gs://cclebams/wes/CDS-rLadW7.bam',
 'gs://cclebams/wes/CDS-ctVpqU.bam',
 'gs://cclebams/wes/CDS-xIv1KJ.bam',
 'gs://cclebams/wes/CDS-ragHOy.bam',
 'gs://cclebams/wes/CDS-4ZOQQF.bam',
 'gs://cclebams/wes/CDS-6xyqy9.bam',
 'gs://cclebams/wes/CDS-Iu8c04.bam',
 'gs://cclebams/wes/CDS-wpXVQk.bam',
 'gs://cclebams/wes/CDS-soTPPi.bam',
 'gs://cclebams/wes/CDS-5ViPeM.bam',
 'gs://cclebams/wes/CDS-EzZEgz.bam',
 'gs://cclebams/wes/CDS-gCSYjV.bam',
 'gs://cclebams/rna/NOS1.bam',
 'gs://cclebams/rna/ONDA10.bam',
 'gs://cclebams/wes/CDS-Ww1LC7.bam',
 'gs://cclebams/wes/CDS-BRxHbu.bam',
 'gs://cclebams/rna/MM466.bam',
 'gs://cclebams/wes/CDS-OxQgBw.bam',
 'gs://cclebams/wes/CDS-RFBAY6.bam',
 'gs://cclebams/wes/CDS-rQIdNN.bam',
 'gs://cclebams/wes/CDS-4l9BUT.bam',
 'gs://cclebams/wes/CDS-Bojgi7.bam',
 'gs://cclebams/wes/CDS-6Fc0S5.bam',
 'gs://cclebams/wes/CDS-DZMoWW.bam',
 'gs://cclebams/wes/CDS-HEoDm7.bam',
 'gs://cclebams/wes/CDS-IGOgCK.bam',
 'gs://cclebams/wes/CDS-AjRIMt.bam',
 'gs://cclebams/wes/CDS-G1sVsw.bam',
 'gs://cclebams/rna/NCCIT.bam',
 'gs://cclebams/wes/CDS-8mpXJa.bam',
 'gs://cclebams/wes/CDS-mtMTts.bam',
 'gs://cclebams/wes/CDS-75psAH.bam',
 'gs://cclebams/wes/CDS-LnV7QY.bam',
 'gs://cclebams/wes/CDS-ydPJEM.bam',
 'gs://cclebams/wes/CDS-tPR3fn.bam',
 'gs://cclebams/wes/CDS-PYWxsh.bam',
 'gs://cclebams/wes/CDS-wWwBMZ.bam',
 'gs://cclebams/wes/CDS-frzvLf.bam',
 'gs://cclebams/wes/CDS-wlTAAF.bam',
 'gs://cclebams/wes/CDS-UVxUrF.bam',
 'gs://cclebams/wes/CDS-x21VqU.bam',
 'gs://cclebams/wes/CDS-tYXity.bam',
 'gs://cclebams/rna/RO82W1.bam',
 'gs://cclebams/wes/CDS-Md89va.bam',
 'gs://cclebams/wes/CDS-mazUYU.bam',
 'gs://cclebams/wes/CDS-wbPtTZ.bam',
 'gs://cclebams/wes/CDS-gsqqAz.bam',
 'gs://cclebams/wes/CDS-ik526H.bam',
 'gs://cclebams/wes/CDS-83LhEq.bam',
 'gs://cclebams/wes/CDS-MhXQX3.bam',
 'gs://cclebams/wes/CDS-Awmxa5.bam',
 'gs://cclebams/wes/CDS-z8Bvmk.bam',
 'gs://cclebams/wes/CDS-HOVBCg.bam',
 'gs://cclebams/wes/CDS-0qPmaJ.bam',
 'gs://cclebams/wes/CDS-Pkk9e2.bam',
 'gs://cclebams/wes/CDS-6EyvRQ.bam',
 'gs://cclebams/wes/CDS-nYIBWR.bam',
 'gs://cclebams/wes/CDS-NXnWiI.bam',
 'gs://cclebams/wes/CDS-8aHSii.bam',
 'gs://cclebams/wes/CDS-Eh7ost.bam',
 'gs://cclebams/wes/CDS-uGZguG.bam',
 'gs://cclebams/wes/CDS-CRPZeK.bam',
 'gs://cclebams/wes/CDS-n7Fqfe.bam',
 'gs://cclebams/wes/CDS-1PXzlf.bam',
 'gs://cclebams/wes/CDS-b9sdh9.bam',
 'gs://cclebams/wes/CDS-LNTGnh.bam',
 'gs://cclebams/wes/CDS-M1sAGX.bam',
 'gs://cclebams/wes/CDS-dNVjOc.bam',
 'gs://cclebams/wes/CDS-fLsYaB.bam',
 'gs://cclebams/wes/CDS-loy9vi.bam',
 'gs://cclebams/wes/CDS-nOKbmw.bam',
 'gs://cclebams/wes/CDS-SO3AhH.bam',
 'gs://cclebams/wes/CDS-r5Ym7C.bam',
 'gs://cclebams/rna/MP41.bam',
 'gs://cclebams/rna/OZ.bam',
 'gs://cclebams/wes/CDS-Hv0i3y.bam',
 'gs://cclebams/wes/CDS-Hw6KuA.bam',
 'gs://cclebams/rna/TANOUE.bam',
 'gs://cclebams/rna/U2973.bam',
 'gs://cclebams/wes/CDS-allHxr.bam',
 'gs://cclebams/wes/CDS-TDblpN.bam',
 'gs://cclebams/wes/CDS-3jIdRa.bam',
 'gs://cclebams/wes/CDS-ck9vpG.bam',
 'gs://cclebams/wes/CDS-EpURcL.bam',
 'gs://cclebams/rna/TL1.bam',
 'gs://cclebams/wes/CDS-2Q2Kia.bam',
 'gs://cclebams/wes/CDS-LUm1Vn.bam',
 'gs://cclebams/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/CHL-1/v2/CHL-1.bam',
 'gs://cclebams/wes/CDS-96DdrP.bam',
 'gs://cclebams/wes/CDS-TSDUCK.bam',
 'gs://cclebams/wes/CDS-ZMsoXe.bam',
 'gs://cclebams/wes/CDS-8yHnJv.bam',
 'gs://cclebams/wes/CDS-7JWzyA.bam',
 'gs://cclebams/wes/CDS-bPT1F0.bam',
 'gs://cclebams/wes/CDS-ycD9px.bam',
 'gs://cclebams/wes/CDS-Ql8GJZ.bam',
 'gs://cclebams/wes/CDS-g0KUGN.bam',
 'gs://cclebams/rna/TY82.bam',
 'gs://cclebams/rna/WSUFSCCL.bam',
 'gs://cclebams/wes/CDS-2lAFkD.bam',
 'gs://cclebams/rna/MM485.bam',
 'gs://cclebams/wes/CDS-XJDBDj.bam',
 'gs://cclebams/wes/CDS-PyELSk.bam',
 'gs://cclebams/wes/CDS-kAARUi.bam',
 'gs://cclebams/wes/CDS-rUs3FP.bam',
 'gs://cclebams/wes/CDS-Uru0Mh.bam',
 'gs://cclebams/wes/CDS-aXqwpM.bam',
 'gs://cclebams/wes/CDS-l1OClV.bam',
 'gs://cclebams/wes/CDS-lSpYo6.bam',
 'gs://cclebams/wes/CDS-5IcijG.bam',
 'gs://cclebams/wes/CDS-RnsUHX.bam',
 'gs://cclebams/wes/CDS-9zidMf.bam',
 'gs://cclebams/rna/KKU055.bam',
 'gs://cclebams/wes/CDS-XqaEOX.bam',
 'gs://cclebams/rna/NH12.bam',
 'gs://cclebams/wes/CDS-aGMcvr.bam',
 'gs://cclebams/wes/CDS-QtTdY6.bam',
 'gs://cclebams/wes/CDS-8sQWae.bam',
 'gs://cclebams/wes/CDS-jHqXGP.bam',
 'gs://cclebams/wes/CDS-LVeuLY.bam',
 'gs://cclebams/wes/CDS-2xSJmZ.bam',
 'gs://cclebams/rna/OSC20.bam',
 'gs://cclebams/wes/CDS-5PXB9Y.bam',
 'gs://cclebams/wes/CDS-u1AlUI.bam',
 'gs://cclebams/wes/9505BIK.bam',
 'gs://cclebams/wes/CDS-mGHY2S.bam',
 'gs://cclebams/wes/CDS-49xzNU.bam',
 'gs://cclebams/wes/CDS-3DHwSX.bam',
 'gs://cclebams/wes/CDS-V2ZEuP.bam',
 'gs://cclebams/wes/CDS-VCuHjJ.bam',
 'gs://cclebams/wes/CDS-cmV75B.bam',
 'gs://cclebams/wes/CDS-OnIxUL.bam',
 'gs://cclebams/wes/CDS-HoW111.bam',
 'gs://cclebams/wes/CDS-cMvnjL.bam',
 'gs://cclebams/wes/CDS-dJqQ4g.bam',
 'gs://cclebams/wes/CDS-Ty3mgt.bam',
 'gs://cclebams/wes/CDS-eZg4P8.bam',
 'gs://cclebams/wes/CDS-9JpX07.bam',
 'gs://cclebams/rna/RPMI2650.bam',
 'gs://cclebams/wes/CDS-3FueNQ.bam',
 'gs://cclebams/wes/CDS-O1ShTQ.bam',
 'gs://cclebams/wes/CDS-RxQhcq.bam',
 'gs://cclebams/rna/HDMB03.bam',
 'gs://cclebams/wes/CDS-3VNhFC.bam',
 'gs://cclebams/wes/CDS-sCWLGL.bam',
 'gs://cclebams/wes/CDS-x7srFK.bam',
 'gs://cclebams/wes/CDS-Twv1kD.bam',
 'gs://cclebams/rna/HAC2.bam',
 'gs://cclebams/wes/CDS-qIc5x3.bam',
 'gs://cclebams/wes/CDS-AJMYsd.bam',
 'gs://cclebams/wes/CDS-hOI086.bam',
 'gs://cclebams/wes/CDS-5H2go6.bam',
 'gs://cclebams/wes/CDS-zwAn7G.bam',
 'gs://cclebams/rna/LO68.bam',
 'gs://cclebams/wes/CDS-294bk6.bam',
 'gs://cclebams/wes/CDS-ziEOXJ.bam',
 'gs://cclebams/wes/CDS-VBr00g.bam',
 'gs://cclebams/rna/ULA.bam',
 'gs://cclebams/wes/CDS-dWHWU3.bam',
 'gs://cclebams/rna/MMNK1.bam',
 'gs://cclebams/wes/CDS-H4hPhD.bam',
 'gs://cclebams/wes/CDS-cBOy2Z.bam',
 'gs://cclebams/wes/CDS-NZsio7.bam',
 'gs://cclebams/wes/CDS-5LNjjI.bam',
 'gs://cclebams/wes/CDS-JvOeJK.bam',
 'gs://cclebams/wes/CDS-xKNh7Q.bam',
 'gs://cclebams/wes/CDS-2hGt1N.bam',
 'gs://cclebams/wes/CDS-tgnRyK.bam',
 'gs://cclebams/wes/CDS-Eo5oAR.bam',
 'gs://cclebams/wes/CDS-T10Uph.bam',
 'gs://cclebams/wes/CDS-QE7bdY.bam',
 'gs://cclebams/wes/CDS-1uWUTi.bam']
torm

['gs://cclebams/wes/CDS-eGQYXr.bam',
 'gs://cclebams/wes/CDS-Fz0HXE.bam',
 'gs://cclebams/wes/CDS-4S6juQ.bam',
 'gs://cclebams/wes/CDS-UL1jLm.bam',
 'gs://cclebams/rna/UPCISCC111.bam',
 'gs://cclebams/wes/CDS-gKIdjs.bam',
 'gs://cclebams/rna/TERA2.bam',
 'gs://cclebams/wes/CDS-Hx6zuD.bam',
 'gs://cclebams/wes/CDS-Xgu4mi.bam',
 'gs://cclebams/wes/CDS-kt2Gne.bam',
 'gs://cclebams/wes/CDS-SvzhGj.bam',
 'gs://cclebams/wes/CDS-Pku96X.bam',
 'gs://cclebams/wes/CDS-leyYAD.bam',
 'gs://cclebams/wes/CDS-R6ehaT.bam',
 'gs://cclebams/wes/CDS-cKMeDY.bam',
 'gs://cclebams/wes/CDS-I7bMcd.bam',
 'gs://cclebams/wes/CDS-GnBdHN.bam',
 'gs://cclebams/wes/CDS-PHI8VT.bam',
 'gs://cclebams/wes/CDS-Y27yfi.bam',
 'gs://cclebams/wes/CDS-sieIuO.bam',
 'gs://cclebams/rna/MM253.bam',
 'gs://cclebams/wes/CDS-3mvYnW.bam',
 'gs://cclebams/rna/PEDS117.bam',
 'gs://cclebams/wes/CDS-AOWMF3.bam',
 'gs://cclebams/wes/CDS-8z476r.bam',
 'gs://cclebams/wes/CDS-qaOoHQ.bam',
 'gs://cclebams/wes/CDS-xCyamv.bam',
 'gs://cclebam

In [71]:
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['size']==val[1])]
    if len(res) >0:
        print(val[0], res['internal_bam_filepath'])
        torm.append(val[0])
        remaining.remove(val[0])

gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam CDS-sukIAT    gs://cclebams/hg38_wes/CDS-sukIAT.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-7JWzyA.hg38.bam CDS-IAZz6r    gs://cclebams/hg38_wes/CDS-IAZz6r.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-n7Fqfe.hg38.bam CDS-bweujf    gs://cclebams/hg38_wes/CDS-bweujf.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-BRxHbu.hg38.bam CDS-zvvAOM    gs://cclebams/hg38_wes/CDS-zvvAOM.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-nby0QM.hg38.bam CDS-Oy6cpj    gs://cclebams/hg38_wes/CDS-Oy6cpj.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-2xSJmZ.hg38.bam CDS-0wO1Np    gs://cclebams/hg38_wes/CDS-0wO1Np.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-5rD8XC.hg38.bam CDS-ML22GG    gs://cclebams/hg38_wes/CDS-ML22GG.hg38.bam
Name: internal_bam_filepath, dtype: object

In [11]:
## removing
a = gcp.rmFiles(torm, dryrun=False)

pressed ctrl+c or command failed


TypeError: object NoneType can't be used in 'await' expression

In [72]:
## preventing remapping of found ones
weswm.update_sample_set("toremap", set(remapable.index) - set(found))

Sample set "toremap" (136 samples) successfully updated.


In [20]:
# loading the remapped version to ccle sample tracker
ss = weswm.get_sample_sets()

In [19]:
sam = weswm.get_samples().loc[ss.loc['toremap'].samples]

In [23]:
e = []
for i in sam["readgroup_ubams"].values.tolist():
    if i is not np.nan:
        e.extend(i)

In [ ]:
gcp.rmFiles(e)

In [28]:
newloc = terra.changeGSlocation("broad-firecloud-ccle/DepMap_WES_CN_hg38", newgs="gs://cclebams/wes/", onlycol=['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index'], entity='sample', keeppath=False, dry_run = False, onlysamples=ss.loc['toremap'].samples)

using the data from broad-firecloud-ccle/DepMap_WES_CN_hg38 sample list
this should only contains gs:// paths otherwise precise columns using "onlycol"
Successfully updated attributes '['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index']' for 136 samples.


In [43]:
newloc = newloc[0].applymap(lambda v: v.replace('.hg38.', '.hg19.'))

In [40]:
h.parrun(['gsutil mv '+val+' '+val.replace('.hg38.', '.hg19.') for val in newloc[0].values.ravel()], 10)

In [51]:
newloc[['hg38_analysis_ready_bam',"hg38_analysis_ready_bam_index"]] =  ccle_refsamples.loc[newloc.index, ['internal_bam_filepath','internal_bai_filepath']]

In [47]:
ccle_refsamples.loc[newloc.index, ['legacy_bam_filepath','legacy_bai_filepath']] = newloc.values

In [54]:
weswm.update_sample_attributes(newloc)

Successfully updated attributes '['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index']' for 136 samples.
Hound executing batch upload of 816 records


In [49]:
ccle_refsamples.loc[newloc.index, ['legacy_bam_filepath','legacy_bai_filepath']]

,legacy_bam_filepath,legacy_bai_filepath
CDS-02waxZ,gs://cclebams/wes/CDS-02waxZ.hg19.bam,gs://cclebams/wes/CDS-02waxZ.hg19.bai
CDS-05XWcR,gs://cclebams/wes/CDS-05XWcR.hg19.bam,gs://cclebams/wes/CDS-05XWcR.hg19.bai
CDS-0oUrUO,gs://cclebams/wes/CDS-0oUrUO.hg19.bam,gs://cclebams/wes/CDS-0oUrUO.hg19.bai
CDS-1CreH8,gs://cclebams/wes/CDS-1CreH8.hg19.bam,gs://cclebams/wes/CDS-1CreH8.hg19.bai
CDS-1TxZbx,gs://cclebams/wes/CDS-1TxZbx.hg19.bam,gs://cclebams/wes/CDS-1TxZbx.hg19.bai
...,...,...
CDS-ynIh7L,gs://cclebams/wes/CDS-ynIh7L.hg19.bam,gs://cclebams/wes/CDS-ynIh7L.hg19.bai
CDS-ysSsBG,gs://cclebams/wes/CDS-ysSsBG.hg19.bam,gs://cclebams/wes/CDS-ysSsBG.hg19.bai
CDS-zbdBmn,gs://cclebams/wes/CDS-zbdBmn.hg19.bam,gs://cclebams/wes/CDS-zbdBmn.hg19.bai
CDS-ZiiXfY,gs://cclebams/wes/CDS-ZiiXfY.hg19.bam,gs://cclebams/wes/CDS-ZiiXfY.hg19.bai


In [76]:
## are they very close to a related sample? (yes? drop)

[('gs://cclebams/hg38_wes/CDS-ydPJEM.hg38.bam', 33590349704),
 ('gs://cclebams/hg38_wes/CDS-s7pOQR.hg38.bam', 52259732517),
 ('gs://cclebams/wes/CDS-eGQYXr.bam', 24098037187),
 ('gs://cclebams/hg38_wes/CDS-Pkk9e2.hg38.bam', 16985791742),
 ('gs://cclebams/wes/CDS-Fz0HXE.bam', 25843558785),
 ('gs://cclebams/hg38_wes/CDS-soTPPi.hg38.bam', 15724674947),
 ('gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam', 18181200052),
 ('gs://cclebams/wes/CDS-4S6juQ.bam', 29576566390),
 ('gs://cclebams/wes/CDS-UL1jLm.bam', 25516864764),
 ('gs://cclebams/rna/UPCISCC111.bam', 3821164738),
 ('gs://cclebams/hg38_wes/CDS-KgRznV.hg38.bam', 16089050631),
 ('gs://cclebams/hg38_wes/CDS-txTRwz.hg38.bam', 13238732111),
 ('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217),
 ('gs://cclebams/hg38_wes/CDS-zwAn7G.hg38.bam', 20340862682),
 ('gs://cclebams/hg38_wes/CDS-fs8moU.hg38.bam', 21098867720),
 ('gs://cclebams/wes/CDS-gKIdjs.bam', 16134677521),
 ('gs://cclebams/hg38_wes/CDS-kAARUi.hg38.bam', 1877

In [ ]:
## can we find a related CDS id? (yes? find out if close else drop)
ccle_refsamples[ccle_refsamples.index.isin([i.split('/')[-1].split('.')[0] for i in remaining if "CDS" in i])]

In [78]:
## do we have this cell line already? (investigate, either add or drop)
sizze = [gcp.extractSize(u) for u in gcp.lsFiles([i for i in remaining if "CDS" not in i],'-l')]

listing files in gs


In [89]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples['size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['size'] == a][['stripped_cell_name','arxspan_id']])

------------
('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217)
22799628189.0
           stripped_cell_name  arxspan_id
CDS-nQIkC1            95T1000  ACH-001796
------------
('gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam', 18871404272)
18871265251.0
           stripped_cell_name  arxspan_id
CDS-eyELts          RERFLCAD1  ACH-000791
------------
('gs://cclebams/wes/FARAGE.bam', 25369720311)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/DepMap_CellLine_WES_May2019/RP-1561/Exome/P30OHK_1/v1/P30OHK_1.bam', 23761432371)
23743557362.0
           stripped_cell_name  arxspan_id
CDS-fAtTIx         NCCSTCK140  ACH-000110
------------
('gs://cclebams/wes/RP-1561/Exome/DOTC24510/current/DOTC24510.bam', 25448357213)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/v2/WERIRB1.bam', 27335320368)
27163656004.0


20474056547.0
           stripped_cell_name  arxspan_id
CDS-TunXjE              U2940  ACH-001678
------------
('gs://cclebams/wes/RP-1561/Exome/Ca_Ski/current/Ca_Ski.bam', 21399623326)
21409812881.0
           stripped_cell_name  arxspan_id
CDS-SYO7e7                CJM  ACH-000458
------------
('gs://cclebams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/94T778/v1/94T778.bam', 25473572546)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/RP-1561/Exome/TGBC1TKB/v1/TGBC1TKB.bam', 26496784045)
26348835512.0
           stripped_cell_name  arxspan_id
CDS-naUpJu                RH4  ACH-001765
------------
('gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam', 24761705198)
24786017590.0
           stripped_cell_name  arxspan_id
CDS-j5EzNM            HCC1806  ACH-000624
------------
('gs://cclebams/wgs/G32831/Hs_944.T/v1/Hs_944.T.bam', 138381206764)
138735205256.0
           stri

In [ ]:
gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam'

In [91]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples[~ccle_refsamples['legacy_size'].isna()]['legacy_size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['legacy_size'] == a][['stripped_cell_name','arxspan_id']])

------------
('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217)
22787650593.0
           stripped_cell_name  arxspan_id
CDS-64cEne             LNZ308  ACH-000760
------------
('gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam', 18871404272)
18870960791.0
           stripped_cell_name  arxspan_id
CDS-ChYgr0                EW7  ACH-002119
------------
('gs://cclebams/wes/FARAGE.bam', 25369720311)
25376759084.0
           stripped_cell_name  arxspan_id
CDS-YBCU7c           NCIH1436  ACH-000830
------------
('gs://cclebams/wes/DepMap_CellLine_WES_May2019/RP-1561/Exome/P30OHK_1/v1/P30OHK_1.bam', 23761432371)
23768902747.0
           stripped_cell_name  arxspan_id
CDS-Oez2VA           NCIBL128  ACH-002362
------------
('gs://cclebams/wes/RP-1561/Exome/DOTC24510/current/DOTC24510.bam', 25448357213)
25440914106.0
           stripped_cell_name  arxspan_id
CDS-ZzuUDl          TGBC52TKB  ACH-001862
------------
('gs://cclebams/wes/v2/WERIRB1.bam', 27335320368)
27327135877.0


21412825896.0
           stripped_cell_name  arxspan_id
CDS-rn0tZ5             UMUC11  ACH-001413
------------
('gs://cclebams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/94T778/v1/94T778.bam', 25473572546)
25465781084.0
           stripped_cell_name  arxspan_id
CDS-RM7bNy              LN319  ACH-000128
------------
('gs://cclebams/wes/RP-1561/Exome/TGBC1TKB/v1/TGBC1TKB.bam', 26496784045)
26491448378.0
           stripped_cell_name  arxspan_id
CDS-oejd1Z               C4II  ACH-001335
------------
('gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam', 24761705198)
24759907179.0
           stripped_cell_name  arxspan_id
CDS-nymcCj               LAN2  ACH-001354
------------
('gs://cclebams/wgs/G32831/Hs_944.T/v1/Hs_944.T.bam', 138381206764)
138486776916.0
           stripped_cell_name  arxspan_id
CDS-rpqo0J             SNU475  ACH-000422
------------
('gs://cclebams/wes/RP-1561/Exome/TASK1/v1/TASK1.bam', 22239375545)
22238628098.0
           stripped_ce

In [101]:
## match md5 to bam files
md5s = [u for u in i if u.endswith('.md5')]
for val in md5s:
    rres = ccle_refsamples[(ccle_refsamples.internal_bam_filepath==val[:-4])&(ccle_refsamples.md5_hash.isna())]
    if len(rres)>0:
        print(rres)

In [102]:
## delete md5s
gcp.rmFiles(md5s, dryrun=False)

In [65]:
## missing tissue annot
set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id)

{'ACH-000049',
 'ACH-000064',
 'ACH-000170',
 'ACH-000195',
 'ACH-000471',
 'ACH-000494',
 'ACH-000561',
 'ACH-000642',
 'ACH-000931',
 'ACH-001046',
 'ACH-001078',
 'ACH-001129',
 'ACH-001142',
 'ACH-001151',
 'ACH-001182',
 'ACH-001187',
 'ACH-001208',
 'ACH-001227',
 'ACH-001293',
 'ACH-001310',
 'ACH-001338',
 'ACH-001349',
 'ACH-001385',
 'ACH-001393',
 'ACH-001400',
 'ACH-001429',
 'ACH-001437',
 'ACH-001438',
 'ACH-001449',
 'ACH-001493',
 'ACH-001502',
 'ACH-001512',
 'ACH-001533',
 'ACH-001537',
 'ACH-001543',
 'ACH-001547',
 'ACH-001553',
 'ACH-001560',
 'ACH-001562',
 'ACH-001605',
 'ACH-001636',
 'ACH-001641',
 'ACH-001642',
 'ACH-001648',
 'ACH-001662',
 'ACH-001669',
 'ACH-001670',
 'ACH-001672',
 'ACH-001676',
 'ACH-001679',
 'ACH-001686',
 'ACH-001687',
 'ACH-001688',
 'ACH-001693',
 'ACH-001696',
 'ACH-001699',
 'ACH-001703',
 'ACH-001705',
 'ACH-001707',
 'ACH-001708',
 'ACH-001818',
 'ACH-001820',
 'ACH-001828',
 'ACH-001854',
 'ACH-001855',
 'ACH-001863',
 'ACH-0019

In [106]:
## missing name/arxspan
ccle_refsamples[ccle_refsamples['arxspan_id'].isna() | ccle_refsamples['stripped_cell_name'].isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [43]:
# clean annots:
set(ccle_refsamples.primary_disease)

{'Adrenal Cancer',
 'Bile Duct Cancer',
 'Bladder Cancer',
 'Bone Cancer',
 'Brain Cancer',
 'Breast Cancer',
 'Cervical Cancer',
 'Colon/Colorectal Cancer',
 'Embryonal Cancer',
 'Endometrial/Uterine Cancer',
 'Engineered',
 'Esophageal Cancer',
 'Eye Cancer',
 'Gallbladder Cancer',
 'Gastric Cancer',
 'Head and Neck Cancer',
 'Kidney Cancer',
 'Leukemia',
 'Liposarcoma',
 'Liver Cancer',
 'Lung Cancer',
 'Lymphoma',
 'Myeloma',
 'Neuroblastoma',
 'Ovarian Cancer',
 'Pancreatic Cancer',
 'Prostate Cancer',
 'Rhabdoid',
 'Sarcoma',
 'Skin Cancer',
 'Teratoma',
 'Thymic Cancer',
 'Thyroid Cancer',
 'Unknown',
 'Vulvar',
 'esophagus_squamous',
 nan,
 'normal'}

In [42]:
set(ccle_refsamples.primary_site)

{'Colon',
 'Embryonal',
 'Esophageal Cancer',
 'Placenta',
 'Thymus',
 'abdomen',
 'ascites',
 'autonomic_ganglia',
 'biliary_tract',
 'bone',
 'bone_marrow',
 'brain',
 'breast',
 'central_nervous_system',
 'cervix',
 'endometrium',
 'engineered',
 'eye',
 'fibroblast',
 'haematopoietic_and_lymphoid_tissue',
 'kidney',
 'large_intestine',
 'liver',
 'lung',
 'lymph_node',
 nan,
 'oesophagus',
 'ovary',
 'pancreas',
 'pericardial_effusion',
 'pleura',
 'pleural_effusion',
 'prostate',
 'salivary_gland',
 'skin',
 'small_intestine',
 'soft_tissue',
 'spleen',
 'stomach',
 'thyroid',
 'upper_aerodigestive_tract',
 'urinary_tract',
 'uvea',
 'vul'}

In [41]:
set(ccle_refsamples.age)

{'0',
 '0.1',
 '0.25',
 '0.33',
 '0.5',
 '0.75',
 '0.8',
 '1',
 '1.4',
 '1.6',
 '1.67',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '2.6',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '3.5',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '92',
 'Adult',
 'C',
 'Children',
 'Embryo',
 'Fetus',
 'U'}

In [82]:
ccle_refsamples[ccle_refsamples.age.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [39]:
set(ccle_refsamples.subtype)

{'ALL',
 'AML',
 'ATL',
 'ATRT',
 'Acute Lymphoblastic Leukemia (ALL), B-cell',
 'Adenocarcinoma, clear cell',
 'B-cell',
 'B-cell, Non-Hodgkins, Burkitts',
 'Breast Ductal Carcinoma',
 'CLL',
 'CML',
 'Carcinoma',
 'Cholangiocarcinoma',
 'Engineered',
 'Ewing sarcoma',
 'Ewing_sarcoma',
 'Ewings Sarcoma',
 'Glioblastoma',
 'MMMT',
 'MPNST',
 'Medulloblastoma',
 'Melanoma',
 'Mesothelioma',
 'Mixed Uterine Adenocarcinoma and Rhabdomyosarcoma',
 'NSCLC',
 'Neuroblastoma',
 'Ovarian carcinoma',
 'PNET',
 'RMS',
 'SCCOHT',
 'SCLC',
 'Small Cell Lung Cancer (SCLC)',
 'Squamous Cell Carcinoma',
 'Testicular',
 'Uveal Melanoma',
 'acute_biphenotypic_leukemia',
 'adrenal_carcinoma',
 'bladder_carcinoma',
 'breast_adenocarcinoma',
 'breast_carcinoma',
 'breast_ductal_carcinoma',
 'brenner_tumor',
 'caecum_adenocarcinoma',
 'cervical_adenocarcinoma',
 'cervical_carcinoma',
 'cervical_squamous',
 'cholangiocarcinoma',
 'chondrosarcoma',
 'chordoma',
 'choriocarcinoma',
 'clear_cell_carcinoma',
 

In [38]:
set(ccle_refsamples.origin)

{'adrenal_cortex',
 'bile_duct',
 'bladder',
 'blood',
 'bone',
 'breast',
 'central_nervous_system',
 'cervix',
 'colorectal',
 'embryo',
 'engineered',
 'engineered_blood',
 'engineered_breast',
 'engineered_central_nervous_system',
 'engineered_kidney',
 'engineered_lung',
 'engineered_ovary',
 'engineered_prostate',
 'engineered_soft_tissue',
 'esophagus',
 'eye',
 'fibroblast',
 'gastric',
 'kidney',
 'liver',
 'lung',
 'lymphocyte',
 nan,
 'ovary',
 'pancreas',
 'peripheral_nervous_system',
 'plasma_cell',
 'prostate',
 'skin',
 'soft_tissue',
 'thyroid',
 'unknown',
 'upper_aerodigestive',
 'urinary_tract',
 'uterus'}

In [87]:
set(ccle_refsamples[ccle_refsamples.source.isna()].arxspan_id)

{'ACH-000004',
 'ACH-000007',
 'ACH-000010',
 'ACH-000019',
 'ACH-000047',
 'ACH-000122',
 'ACH-000150',
 'ACH-000157',
 'ACH-000195',
 'ACH-000200',
 'ACH-000217',
 'ACH-000247',
 'ACH-000261',
 'ACH-000278',
 'ACH-000283',
 'ACH-000302',
 'ACH-000304',
 'ACH-000312',
 'ACH-000316',
 'ACH-000337',
 'ACH-000398',
 'ACH-000419',
 'ACH-000434',
 'ACH-000436',
 'ACH-000452',
 'ACH-000458',
 'ACH-000461',
 'ACH-000466',
 'ACH-000473',
 'ACH-000487',
 'ACH-000509',
 'ACH-000511',
 'ACH-000514',
 'ACH-000537',
 'ACH-000544',
 'ACH-000557',
 'ACH-000577',
 'ACH-000596',
 'ACH-000608',
 'ACH-000655',
 'ACH-000662',
 'ACH-000672',
 'ACH-000677',
 'ACH-000698',
 'ACH-000736',
 'ACH-000740',
 'ACH-000757',
 'ACH-000758',
 'ACH-000767',
 'ACH-000800',
 'ACH-000823',
 'ACH-000825',
 'ACH-000837',
 'ACH-000840',
 'ACH-000866',
 'ACH-000868',
 'ACH-000877',
 'ACH-000898',
 'ACH-000901',
 'ACH-000912',
 'ACH-000921',
 'ACH-000931',
 'ACH-000946',
 'ACH-000993',
 'ACH-001046',
 'ACH-001075',
 'ACH-0010

In [54]:
ccle_refsamples[ccle_refsamples.sex.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [17]:
set(ccle_refsamples.baits)

{'AGILENT', 'ICE', nan, 'polyA'}

In [36]:
ccle_refsamples.loc[k, upd]

participant_id                             PT-W5WB8L8R
cellosaurus_id                               CVCL_0217
parent_cell_line                                   NaN
sex                                               Male
matched_normal                                     NaN
age                                                  0
primary_site        haematopoietic_and_lymphoid_tissue
primary_disease                               Leukemia
subtype                                            AML
subsubtype                                         NaN
origin                                           blood
Name: CDS-2FgC1U, dtype: object

In [1]:
k

NameError: name 'k' is not defined

In [79]:
upd = ['participant_id', 'cellosaurus_id', 'parent_cell_line', 'sex',
       'matched_normal', 'age', 'primary_site', 'primary_disease', 'subtype',
       'subsubtype', 'origin']
for k in ["CDS-yYeDem",
            "CDS-OT217E",
            "CDS-a8BCnW",
            "CDS-zeFugQ",
            "CDS-GOI63U",
            "CDS-ldvG6N",
            "CDS-LFK9QA",
            "CDS-SdYhEt",
            "CDS-gyyVs5",
            "CDS-IgYH22",
            "CDS-PoGjEx",
            "CDS-J80Eb1",
            "CDS-Hb7hHv",
            "CDS-3VSkPT",
            "CDS-2FgC1U"]:
    res = ccle_refsamples[(ccle_refsamples.arxspan_id==ccle_refsamples.loc[k,"arxspan_id"]) &(~ccle_refsamples.age.isna()) & (ccle_refsamples.index!=k)]
    if len(res)>0:
        ccle_refsamples.loc[k, upd] = res.iloc[0][upd].values
    else:
        print(k)

CDS-a8BCnW
CDS-zeFugQ
CDS-IgYH22
CDS-3VSkPT


In [83]:
upd = ['participant_id', 'cellosaurus_id', 'parent_cell_line', 'sex',
       'matched_normal', 'age', 'primary_site', 'primary_disease', 'subtype',
       'subsubtype', 'origin']
for k, v in ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna() | ccle_refsamples['source'].isna()].iterrows():
    res = ccle_refsamples[(ccle_refsamples.arxspan_id==v.arxspan_id) & (ccle_refsamples.index!=k)]
    if len(res)>0:
        ccle_refsamples.loc[k, upd] = res.iloc[0][upd].values

In [88]:
## missing tissue annot
len(set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id))

286

In [33]:
ccle_refsamples[(ccle_refsamples.arxspan_id==val.arxspan_id) &(~ccle_refsamples.age.isna())].iloc[0][upd].values

array(['PT-y3RbI7uD', 'CVCL_Y549', nan, 'Male', nan, 'Adult',
       'haematopoietic_and_lymphoid_tissue', 'Leukemia', 'CLL', 'b_cell',
       nan], dtype=object)

In [89]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

## PART 4

In [261]:
wm.disable_hound().update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash', 'internal_bam_filepath', 'internal_bam_filepath', 'size', 'crc32c_hash', 'md5_hash']' for 412 samples.


In [262]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')

In [490]:
a = wm.get_samples()

In [300]:
ss= h.dups(ccle_refsamples['size'].tolist()+ccle_refsamples['legacy_size'].tolist())

In [307]:
issue = {val:[] for val in ss}
recom = []
for val in ss:
    for k,v in ccle_refsamples[(ccle_refsamples['size']==val)].iterrows():
        if v.size!= gcp.extractSize(gcp.lsFiles([v.internal_bam_filepath],"-l")[0]):
            recom.append(k)
        else:
            issue[val].append(k)
    for k,v in ccle_refsamples[(ccle_refsamples['legacy_size']==val)].iterrows():
        if v.legacy_size!= gcp.extractSize(gcp.lsFiles([v.legacy_bam_filepath],"-l")[0]):
            recom.append(k)    
        else:
            issue[val].append(k)

listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs


KeyboardInterrupt: 

In [422]:
e = gcp.lsFiles(a.legacy_bam_filepath.tolist(),"-L")

listing files in gs


In [418]:
wm.update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash']' for 15 samples.
Hound executing batch upload of 150 records


In [321]:
for k, val in ccle_refsamples[(ccle_refsamples.datatype=='rna')&(~ccle_refsamples.internal_bam_filepath.str.contains("/rnasq_hg38",na=True))].internal_bam_filepath.iteritems():
    rn = "gs://cclebams/rna/"+'/'.join(val.split('/')[3:])
    print(rn)
    rni = rn[:-1]+'i'
    vali = val[:-1]+"i"
    #! gsutil cp $val $rn
    #! gsutil cp $vali $rni
    ccle_refsamples.loc[k, "internal_bam_filepath"] = rn
    ccle_refsamples.loc[k, "internal_bai_filepath"] = rni
    ccle_refsamples.loc[k, "size"] = gcp.extractSize(gcp.lsFiles([rn], '-l')[0],)[1]
    ccle_refsamples.loc[k, "crc32c_hash"] = gcp.extractHash(gcp.lsFiles([rn], '-L')[0])[1]
    ccle_refsamples.loc[k, "sequencing_date"]= seq.getBamDate([rn])[0]
    print(rn)
    break

gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam [Content-Type=application/octet-stream]...
/ [1 files][  4.8 GiB/  4.8 GiB]                                                
Operation completed over 1 objects/4.8 GiB.                                      
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bai [Content-Type=application/octet-stream]...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation completed over 1 objects/6.5 MiB.                                      
listing files in gs
listing files in gs
gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
gs://cclebams/rna/rna/Getz_IBM_CellLine_RNAseq_June2020/RP-1561/RNA/CORL26/v1/CORL26.bam
Copying gs://cclebams/rna/Getz_IBM_CellLine_RNAseq_June2020

KeyboardInterrupt: 

In [377]:
ccle_refsamples.loc[a[a.legacy_bam_filepath.isna()].index].internal_bam_filepath

sample_id
CDS-2FgC1U    gs://cclebams/rnasq_hg38/CDS-2FgC1U.Aligned.so...
CDS-3VSkPT    gs://cclebams/rnasq_hg38/CDS-3VSkPT.Aligned.so...
CDS-57hTZY    gs://cclebams/rnasq_hg38/CDS-57hTZY.Aligned.so...
CDS-6HLVzC    gs://cclebams/rnasq_hg38/CDS-6HLVzC.Aligned.so...
CDS-6Je1tY    gs://cclebams/rnasq_hg38/CDS-6Je1tY.Aligned.so...
CDS-a8BCnW    gs://cclebams/rnasq_hg38/CDS-a8BCnW.Aligned.so...
CDS-bmH4HU    gs://cclebams/rnasq_hg38/CDS-bmH4HU.Aligned.so...
CDS-FMkkVj    gs://cclebams/rnasq_hg38/CDS-FMkkVj.Aligned.so...
CDS-GOI63U    gs://cclebams/rnasq_hg38/CDS-GOI63U.Aligned.so...
CDS-gyyVs5    gs://cclebams/rnasq_hg38/CDS-gyyVs5.Aligned.so...
CDS-Hb7hHv    gs://cclebams/rnasq_hg38/CDS-Hb7hHv.Aligned.so...
CDS-IgYH22    gs://cclebams/rnasq_hg38/CDS-IgYH22.Aligned.so...
CDS-j3NCLS    gs://cclebams/rnasq_hg38/CDS-j3NCLS.Aligned.so...
CDS-J80Eb1    gs://cclebams/rnasq_hg38/CDS-J80Eb1.Aligned.so...
CDS-ldvG6N    gs://cclebams/rnasq_hg38/CDS-ldvG6N.Aligned.so...
CDS-LFK9QA    gs://cclebams/rn

In [212]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size']

CDS-6dPoX7    13488934939
CDS-zmiamL     7606219711
CDS-3wjudI     8571265034
CDS-uUP01E     8733556513
CDS-UFAuQl     8852095963
                 ...     
CDS-PoGjEx     7813216031
CDS-J80Eb1     4940855845
CDS-Hb7hHv     4613838702
CDS-3VSkPT     5219564814
CDS-2FgC1U     7056679221
Name: size, Length: 1516, dtype: int64

In [225]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-l')]

ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-L')]

listing files in gs
listing files in gs


In [156]:
ccle_refsamples[ccle_refsamples.datatype.isin(['wgs'])][["internal_bam_filepath", "internal_bai_filepath", "md5_hash","legacy_bam_filepath","legacy_bai_filepath",'legacy_size', 'legacy_crc32c_hash']]

,internal_bam_filepath,internal_bai_filepath,md5_hash,legacy_bam_filepath,legacy_bai_filepath,legacy_size,legacy_crc32c_hash
CDS-uhGMmZ,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bai,6136551f350a8e8b6b2ca478ac951d49,gs://cclebams/wgs/G16444/G-401/v1/G-401.bam,gs://cclebams/wgs/G16444/G-401/v1/G-401.bai,1.834687e+11,puzdUg==
CDS-vNNLAr,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bai,470c011136c545e5eea12cf71a2910c9,gs://cclebams/wgs/G32831/G-402/v2/G-402.bam,gs://cclebams/wgs/G32831/G-402/v2/G-402.bai,1.428883e+11,08KlNA==
CDS-KoNTMz,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bai,9d00ef2ecef3854d16d3693733760507,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bam,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bai,1.651190e+11,NmAKsw==
CDS-Wmm2y2,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bai,f81251027a48bf0dd030260c1b2d7ebf,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bam,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bai,1.860679e+11,nQhYUA==
CDS-IKutdC,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bai,de1720234eaa10b3ee38fa69cc4f0649,gs://cclebams/wgs/G32831/A-204/v3/A-204.bam,gs://cclebams/wgs/G32831/A-204/v3/A-204.bai,1.513434e+11,c1qSGg==
...,...,...,...,...,...,...,...
CDS-lEoO4A,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,c5368347afae1031645835742185b799,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,8.854451e+10,tFgKDw==
CDS-lZD981,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,35e4b81c789393791ad5c1e2c70cfd1b,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,7.308927e+10,sbmvbA==
CDS-M9kJ4N,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,eb51988decdf6e53473006d833d4fce0,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,7.108076e+10,LdAyfw==
CDS-bNHw6E,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,c1cb0522c5ba22df53e1b684e6f12a64,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,7.505758e+10,NHvcFg==
